##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Using TensorBoard in Notebooks

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorboard/blob/master/docs/tensorboard_in_notebooks.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/tensorboard/blob/master/docs/tensorboard_in_notebooks.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/tensorboard/docs/tensorboard_in_notebooks.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

TensorBoard can be used directly within notebook experiences such as [Colab](https://colab.research.google.com/) and [Jupyter](https://jupyter.org/). This can be helpful for sharing results, integrating TensorBoard into existing workflows, and using TensorBoard without installing anything locally.

## Setup

Start by installing TF 2.0 and loading the TensorBoard notebook extension:

**For Jupyter users:** If you’ve installed Jupyter and TensorBoard into
the same virtualenv, then you should be good to go. If you’re using a
more complicated setup, like a global Jupyter installation and kernels
for different Conda/virtualenv environments, then you must ensure that
the `tensorboard` binary is on your `PATH` inside the Jupyter notebook
context. One way to do this is to modify the `kernel_spec` to prepend
the environment’s `bin` directory to `PATH`, [as described here][1].

[1]: https://github.com/ipython/ipykernel/issues/395#issuecomment-479787997


**For Docker users:** In case you are running a [Docker](https://docs.docker.com/install/) image of [Jupyter Notebook server using TensorFlow's nightly](https://www.tensorflow.org/install/docker#examples_using_cpu-only_images), it is necessary to expose not only the notebook's port, but the TensorBoard's port. Thus, run the container with the following command:

```
docker run -it -p 8888:8888 -p 6006:6006 \
tensorflow/tensorflow:nightly-py3-jupyter 
```

where the `-p 6006` is the default port of TensorBoard. This will allocate a port for you to run one TensorBoard instance. To have concurrent instances, it is necessary to allocate more ports. Also, pass `--bind_all` to `%tensorboard` to expose the port outside the container.

In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

Import TensorFlow, datetime, and os:

In [3]:
import tensorflow as tf
import datetime, os

## TensorBoard in notebooks

Download the [FashionMNIST](https://github.com/zalandoresearch/fashion-mnist) dataset and scale it:

In [4]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

4423680/4422102 [==============================] - 0s 0us/step


Create a very simple model:

In [5]:
def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

Train the model using Keras and the TensorBoard callback:

In [6]:
def train_model():
  
  model = create_model()
  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(x=x_train, 
            y=y_train, 
            epochs=5, 
            validation_data=(x_test, y_test), 
            callbacks=[tensorboard_callback])

train_model()

Epoch 1/5
   1/1875 [..............................] - ETA: 0s - loss: 2.4835 - accuracy: 0.0938WARNING:tensorflow:From C:\Users\hba986\.conda\envs\DeepLearning\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
1875/1875 [==============================] - 5s 2ms/step - loss: 0.4946 - accuracy: 0.8238 - val_loss: 0.4335 - val_accuracy: 0.8409
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3816 - accuracy: 0.8597 - val_loss: 0.3874 - val_accuracy: 0.8585
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3474 - accuracy: 0.8719 - val_loss: 0.3613 - val_accuracy: 0.8702
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3290 - accuracy: 0.8785 - val_loss: 0.3419 - val_accuracy: 0.8781
Epoch 5/5
1875/1875 [=======================

Start TensorBoard within the notebook using [magics](https://ipython.readthedocs.io/en/stable/interactive/magics.html):

In [7]:
%tensorboard --logdir logs

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/notebook_tensorboard.png?raw=1"/> -->

You can now view dashboards such as scalars, graphs, histograms, and others. Some dashboards are not available yet in Colab (such as the profile plugin).

The `%tensorboard` magic has exactly the same format as the TensorBoard command line invocation, but with a `%`-sign in front of it.

You can also start TensorBoard before training to monitor it in progress:

In [8]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 11888), started 0:01:39 ago. (Use '!kill 11888' to kill it.)

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/notebook_tensorboard_two_runs.png?raw=1"/> -->

The same TensorBoard backend is reused by issuing the same command. If a different logs directory was chosen, a new instance of TensorBoard would be opened. Ports are managed automatically. 

Start training a new model and watch TensorBoard update automatically every 30 seconds or refresh it with the button on the top right:

In [9]:
train_model()

Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.4954 - accuracy: 0.8228 - val_loss: 0.4494 - val_accuracy: 0.8365
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3801 - accuracy: 0.8620 - val_loss: 0.3826 - val_accuracy: 0.8611
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3503 - accuracy: 0.8708 - val_loss: 0.3664 - val_accuracy: 0.8672
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3303 - accuracy: 0.8781 - val_loss: 0.3536 - val_accuracy: 0.8746
Epoch 5/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.3117 - accuracy: 0.8849 - val_loss: 0.3466 - val_accuracy: 0.8757


You can use the `tensorboard.notebook` APIs for a bit more control:

In [10]:
from tensorboard import notebook
notebook.list() # View open TensorBoard instances

Known TensorBoard instances:
  - port 6006: logdir logs (started 0:03:08 ago; pid 11888)


In [11]:
# Control TensorBoard display. If no port is provided, 
# the most recently launched TensorBoard is used
notebook.display(port=6006, height=1000) 

Selecting TensorBoard with logdir logs (started 0:03:10 ago; port 6006, pid 11888).


<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/notebook_tensorboard_tall.png?raw=1"/> -->